# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
np.set_printoptions(precision=7)

In [3]:
data = pd.io.stata.read_stata('lakisha_aer.dta')

In [4]:
data[['race','call']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 2 columns):
race    4870 non-null object
call    4870 non-null float32
dtypes: float32(1), object(1)
memory usage: 95.1+ KB


In [19]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [20]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [21]:
data[['race','call']].head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


## What test is appropriate for this problem?
In this problem, we are testing and estimating the difference between two population proportions, which are the probability of resumes with black-sounding and white-sounding names receiving calls from employers. Since the dataset is large and robust, the test statistic would be z-statistics. The test statistic can be expressed as:

$$z = \frac{\left( \hat{p}_W - \hat{p}_B \right) - 0}{\sqrt{\hat{p} (1 - \hat{p)} \left( \frac{1}{n_W} + \frac{1}{n_B}\right)}}$$

where:

$$ \hat{p}_B = \frac{{x}_B}{{n}_B} \space\space and \space\space\space \hat{p}_W = \frac{{x}_W}{{n}_W}$$

With x representing the sum of successes, the pooled proportion estimate is defined as:

$$\hat{p} = \frac{x_W + x_B}{n_W + n_B}$$

and the confidence interval:

$$\left( \hat{p}_W - \hat{p}_B \right) \pm z_\frac{a}{2} \sqrt{\frac{\hat{p}_W \left( 1- \hat{p}_W \right)}{n_W} + \frac{\hat{p}_B \left( 1- \hat{p}_B \right)}{n_B}}$$

The null hypotheses:

$$H_0: p_B = p_W$$

The confidence level (α) is 0.05. 

In [6]:
#compute the variables in the equations and calculate the z score and p-value
w = data[['call']][data.race=='w']
b = data[['call']][data.race=='b']

n_w = len(w)
n_b = len(b)

x_w = np.sum(w.call)
x_b = np.sum(b.call)

phat_w = x_w/n_w
phat_b = x_b/n_b
phat = (x_w+x_b)/(n_w+n_b)

z = (phat_w - phat_b)/np.sqrt(phat*(1-phat)*(1/n_w + 1/n_b))
p_value = stats.norm.sf(abs(z))*2

print("Z score:", z)
print("P-value:", p_value)

Z score: 4.10841215243
P-value: 3.98388683759e-05


In [23]:
#calculate both margin of error and confidence interval
m_o_e = 1.96 * np.sqrt(phat*(1-phat)*(1/n_w + 1/n_b))
confi_int = (phat_w - phat_b) + np.array([-1,1]) * m_o_e
print("Margin of error:", m_o_e)
print("Confidence interval:", confi_int)

Margin of error: 0.0152819123109
Confidence interval: [ 0.0167509  0.0473148]


The z score calculated from the sample data is really high and the p-value is nearly 0. Furthermore, the confidence interval is well above 0. Therefore we can reject the null hypothesis and conclude that there is significant difference in call back rate between black-sounding and white-sounding names.

In [35]:
##two sample boostrap confidence interval:
w_array = w.values.flatten()
b_array = b.values.flatten()

#create function to calculate difference in probability in getting calls
def prop(data):
    phat = np.sum(data)/len(data)
    
    return phat

#bootstrap replicate function
def bootstrap_replicate_1d(data, func):
    #""Generate bootstrap replicate of 1D data.""
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample)

#create many bootstrap replicate as a function:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)
    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)
    return bs_replicates

bs_replicates_w = draw_bs_reps(w_array, prop, 10000)
bs_replicates_b = draw_bs_reps(b_array, prop, 10000)

bs_diff_replicates = bs_replicates_w - bs_replicates_b

#calculate the difference in call probabilities between two name categories
prop_diff = prop(w_array) - prop(b_array)

p_value_bs = np.sum(bs_diff_replicates <= 0) / len(bs_diff_replicates)

#calculate the two-tailed 95% confidence interval at percentile 2.5% and 97.5%
conf_int = np.percentile(bs_diff_replicates,[2.5,97.5])
m_o_e = (np.percentile(bs_diff_replicates,97.5) - np.percentile(bs_diff_replicates,2.5))/2

print('P-value:', p_value_bs)
print('Confidence interval:', conf_int)
print('Margin of error:', m_o_e)

P-value: 0.0
Confidence interval: [ 0.0168378  0.0476386]
Margin of error: 0.0154004106776


By bootstrap approach, the result is the same. P-value is at 0. We can also reject the null hypothesis by bootstrapping.

This analysis shows that the difference in racial sounding names is a significant factor toward getting call from employers. The original study accompanied the data source mentioned that name difference is the most important contributor besides other factors, such as region and experience, that impacted the employment prospect.  